# Lab 6
*In this lab we will, as a class, create the grading script for the final project.*

In [1]:
import librosa
import numpy as np
import sys
import pickle
import time
import IPython.display as ipd
from codec import encode, decode

---
## Import Audio File
*Load in an audio file to test with codec.*

In [2]:
def load_cd_quality_audio(filename):
    audio, sr = librosa.load(filename, sr=44100, dtype='float_')
    max_int_value = 2**15 - 1
    audio *= max_int_value     # get audio signal frm lowest max_int_value to highest max_int_value, get it in the right range    audio = audio.astype('int16')
    audio = audio.astype('int16')
    return audio

x = load_cd_quality_audio('taxman.wav')

---
## Runtime
*Encode and decode the audio. Time the processes.*

In [3]:
encodeStartTime = time.time()     # start timer
x_encoded = encode(x)
ert = time.time() - encodeStartTime     # encode run time
print(f"Encode run time: {ert}")

decodeStartTime = time.time()
x_decoded = decode(x)
drt = time.time() - decodeStartTime
print(f"Decode run time: {drt}")

Encode run time: 23.45958662033081
Decode run time: 41.438312292099


---
## Format Check
*Check to make sure the decoded audio is mono 16 bit.*

In [4]:
def check_decoded_output(d):
    if type(d) != np.ndarray:
        print("Your decoded signal is not a numpy array!")
    elif d.dtype != "int16":
        print("Your decoded signal is not 16 bit!")
    elif len(d.shape) != 1:
        print("Your decoded signal is not a 1D vector!")
    else:
        print("Your decoded signal passes the format check.")

check_decoded_output(x_decoded)

Your decoded signal passes the format check.


---
## Compression Ratio
*Compare the sizes of the original and encoded structures.*

In [5]:
def compressionRatio(original, encoded):
    o_str = pickle.dumps(original)
    e_str = pickle.dumps(encoded)
    return sys.getsizeof(o_str) / sys.getsizeof(e_str)

cr = compressionRatio(x, x_encoded)
print(f"Compression ratio: {cr}")

Compression ratio: 1.999972319097454


---
## SNR
*Compare the original signal content to the decoded version.*

In [6]:
def signalToNoise(original, decoded):
    # higher SNR, the better
    
    original = original.astype("float_")
    decoded = decoded.astype("float_")
    
    # force the signals to be the same length
    diff = len(original) - len(decoded)
    if diff < 0:
        decoded = decoded[:diff]
    if diff > 0:
        decoded = np.append(decoded, np.zeros((diff, 1)))
        
    # computs SNR
    signal = np.power(original, 2)
    noise = np.power(original - decoded, 2)
    
    # replace all 0's with the smallest value in Python so we avoid dividing by 0
    signal = np.where(signal == 0, np.finfo(np.float32).eps, signal)
    noise = np.where(noise == 0, np.finfo(np.float32).eps, noise)
    
    return np.mean(10 * np.log10(signal / noise))

snr = signalToNoise(x, x_decoded)
print(f"SNR: {snr}")

SNR: -4.878775252821266


---
## Evaluate Codec
*Print out evalutation of codec. Listen to the results.*

In [7]:
print("Compression Ratio: ", str(round(cr, 4)))
print()
print("Total Runtime: ", str(round(ert + drt, 4)))
print("\tEncode Runtime: ", str(round(ert, 4)))
print("\tDecode Runtime: ", str(round(drt, 4)))
print()
print("SNR: ", str(round(snr, 4)))

Compression Ratio:  2.0

Total Runtime:  64.8979
	Encode Runtime:  23.4596
	Decode Runtime:  41.4383

SNR:  -4.8788


In [ ]:
print("Original Signal")
ipd.Audio(x, rate=44100)

Original Signal


In [ ]:
print("Decoded Signal")
ipd.Audio(x_decoded, rate=44100)